In [1]:
# Installation de la bibliotèque
#pip install mysql-connector-python

In [2]:
# Importation des bibliotques nécessaires au traitement et à la récupération des données
import mysql.connector as sql
import pandas as pd
import numpy
import requests
import re
from sqlalchemy import create_engine

In [3]:
# Connexion à la base de données MySQL local
db_connection = sql.connect(host='localhost',
                         database='dataengineer',
                         user='root',
                         password='root')

In [4]:
# Récupération de la tale "address" afin de traiter le requétage de l'API sur une table Pandas
df = pd.read_sql('SELECT * FROM address', con=db_connection)

In [5]:
# Affichage des premières lignes pour vérifier la présence de toutes les colonnes
df.head()

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,None,None
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,None,None
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,None,None
3,4,6 BD DES ETINES,LE COTEAU,42120,None,None
4,5,35 AV DU 159EME RIA,BRIANCON,5100,None,None


In [6]:
# Création d'une colonne qui regroupe l'adresse complète
df['complete_address'] = df.address + " " + df.postal_code + " " + df.city

In [7]:
# Création d'un fonction pour appeler l'API sur chaque adresse présente dans la table
def API_address(complete_address):
    #Création du lien pour appeler le premier résultat de l'API à une adresse donnée
    link_main = 'https://nominatim.openstreetmap.org/?q='
    link_end = '&format=json&limit=1'
    link = link_main + complete_address.replace(' ' , '+') + link_end
    requete = requests.get(link).json()
    # Si la requête retourne une liste vide, j'élargie la rechrche pour récupérer les coordonnées de la commune via son code postal
    if requete == []:
        result = re.findall(r'(?<!\d)(\d{5})(?!\d)', complete_address) 
        if result == []:
            result = re.findall(r'(?<!\d)(\d{4})(?!\d)', complete_address)
        link = link_main + ''.join(result) + link_end
        points = [ float(requests.get(link).json()[0]['lat']), float(requests.get(link).json()[0]['lon'])]
        return points
    else:
        point = [ float(requests.get(link).json()[0]['lat']), float(requests.get(link).json()[0]['lon'])]   
        return point

In [8]:
# Création d'une colonne stockant les coordonnées dans une liste pour chaque adresse 
df['coordinates'] = df['complete_address'].apply(API_address)

In [9]:
# Génération des valeurs correspondantes pour les colonnes "latitude" et "longitude"
df[['latitude','longitude']] = pd.DataFrame(df.coordinates.tolist(), index= df.index)
df.drop(columns=['complete_address','coordinates'], inplace =True)

In [10]:
df

,address_id,address,city,postal_code,latitude,longitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,43.197642,5.607890
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,48.290905,4.045789
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,48.361340,-4.766800
3,4,6 BD DES ETINES,LE COTEAU,42120,46.021659,4.092120
4,5,35 AV DU 159EME RIA,BRIANCON,5100,49.932415,10.885641
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,48.868459,2.468415
558,602,6 AV JEAN JAURES,FEURS,42110,45.743563,4.226839
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,49.773546,4.708829
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,48.953836,2.292919


In [11]:
df.address_id.iloc[1]

2

In [28]:
cursor=db_connection.cursor()

# Insert DataFrame recrds one by one.
for i in range (len(df.latitude)):
    sql = "UPDATE address SET latitude=%s, longitude=%s WHERE address_id=%s" 
    val = (df.latitude.iloc[i], df.longitude.iloc[i], int(df.address_id.iloc[i]))

    cursor.execute(sql, val)

cursor.close()

# the connection is not autocommitted by default, so we must commit to save our changes
db_connection.commit()

db_connection.close()

In [ ]:
        if len(r):
            query = "UPDATE address SET latitude = %s, longitude = %s WHERE address_id = %s"
            cursor.execute(query, (r[0]['lat'], r[0]['lon'], address_id))

    connection.commit()
    print("Data inserted successfully into dataengineer table using the prepared statement")

except mysql.connector.Error as error:
    print("parameterized query failed", error)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")